In [3]:
pip install openpyxl

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [6]:
import openpyxl
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# for word embedding
import gensim
from gensim.models import Word2Vec
import os

In [5]:
# convert to lowercase, strip and remove punctuations

def preprocess(text):
    text = text.lower() 
    text=text.strip()
    text=re.compile('<.*?>').sub('', text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)
    text = re.sub('\s+', ' ', text)
    text = re.sub(r'\[[0-9]*\]',' ',text)
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 

    text = re.sub(r'\s+',' ',text) 
    return text
# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)
#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()

# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))
# building Word2Vec model
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))
    def fit(self, X, y):
            return self
    
    def transform(self, X):
            return np.array([
                np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                        or [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [13]:
import nltk
nltk.download('omw-1.4')

In [16]:
import joblib
clf = joblib.load('../input/gbdt-model/GradientBoostingClassifier.pkl')
tfidf_vectorizer = joblib.load('../input/gbdt-model/tfidf_vectorizer.pkl')

In [25]:
def find_privacy(name):
    fb = pd.read_csv('../input/order-apps/apps/{}.csv'.format(name),error_bad_lines=False)
    fb['clean_text'] = fb['text'].apply(lambda x: finalpreprocess(x))
    X_test=fb['clean_text'] 
    X_vector=tfidf_vectorizer.transform(X_test)
    y_predict = clf.predict(X_vector)
    fb['label'] = y_predict
    final = fb[['text','label']].reset_index(drop=True)
    p = final[final['label']==1]
    print(len(p))
    with open('{}_privacy.csv'.format(name),'w') as f:
        for i in p['text']:
            f.write('"{}"\n'.format(i))

In [26]:
apps = ["Facebook",
    "Instagram",
"Spotify",
"TikTok",
"Twitter",
"YouTube"]

In [27]:
for i in apps:
    find_privacy(i)